In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate
from os import path
from astropy.io import fits

from spectrum import AtomicLine, Spectrum
from common import list_files, list_directories, DATA_FOLDER, tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# -------------------------------------------------------------------------------------
# -------------------------------------- SOURCES --------------------------------------
# -------------------------------------------------------------------------------------
# Na I:  https://physics.nist.gov/PhysRefData/Handbook/Tables/sodiumtable3.htm
# K  I:  https://physics.nist.gov/PhysRefData/Handbook/Tables/potassiumtable3.htm
# Ca I:  https://physics.nist.gov/PhysRefData/Handbook/Tables/calciumtable3.htm
# Ca II: https://physics.nist.gov/PhysRefData/Handbook/Tables/calciumtable4.htm

NA_I_LINES = [
    AtomicLine('Na I', 5889.950),
    AtomicLine('Na I', 5895.924)
]

K_I_LINES = [
    AtomicLine('K I', 7664.8991),
    AtomicLine('K I', 7698.9645),
    AtomicLine('K I', 5801.75),
    AtomicLine('K I', 5812.15)
]

CA_I_LINES = [
    AtomicLine('Ca I', 6102.722),
    AtomicLine('Ca I', 6122.219)
]

CA_II_LINES = [
    # TODO
]


In [ ]:
def load_spectrum(datafile: str):
    wvl, flux = np.loadtxt(datafile, usecols=(0, 3), unpack=True, skiprows=1)
    target_name = path.basename(datafile).split('_')[0]

    return Spectrum(target_name, wvl, flux)

def load_fits_to_spectrum(target_dir: str):
    sub_spectra = []
    
    for band_dir in list_directories(target_dir):
        for datafile in list_files(band_dir):
            if not datafile.endswith('.fits'):
                print(f'[WARNING]: skipping unexpected datafile: {datafile}')
                continue

            with fits.open(datafile) as hdulist:
                header = hdulist[0].header
                target = header['OBJECT']
                starting_wvl = header['CRVAL1']
                stepsize_wvl = header['CDELT1']

                flux = hdulist[0].data
                wavelength = np.arange(0, len(flux), 1) * stepsize_wvl + starting_wvl
                flux[np.abs(flux) > 1.01 * np.median(flux)] = np.median(flux)

                fig, ax = plt.subplots(1, 1, figsize=(15,3))
                spectrum = Spectrum(target, wavelength, flux).normalize(1, ax)
                sub_spectra.append(spectrum)

    return sub_spectra

# hd23180 = load_fits_to_spectrum('data/fits/HD23180')
# spectra = [load_fits_to_spectrum(target_dir) for target_dir in tqdm(list_directories(path.join(DATA_FOLDER, 'fits')), 'Loading spectra')]

hd23180 = load_spectrum('data/ascii/HD185859/')
spectra = [load_spectrum(datafile) for datafile in tqdm(list_files(path.join(DATA_FOLDER, 'DR4')), 'Loading spectra')]

FileNotFoundError: data/DR4/HD23180_w564_n3_20140923_U.ascii not found.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 6))
na_line_pairs = hd23180.identify_atomic_line(
    NA_I_LINES[0], NA_I_LINES[1],
    axes=axes, draw_expected=True,
    output_difference=True,
    output_radial_velocity=True
)

fig.suptitle(f'Na I atomic line detection for {hd23180.target}')
fig.tight_layout()

tabulate(na_line_pairs, ['Line 1 [Å]', 'Line 2 [Å]', 'Deviation [Å]', 'Radial velocity [km/s]'], tablefmt='html')

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 6))
k_line_pairs = hd23180.identify_atomic_line(
    K_I_LINES[2], K_I_LINES[3],
    axes=axes, draw_expected=True,
    output_difference=True,
    output_radial_velocity=True
)

fig.suptitle(f'K I atomic line detection for {hd23180.target}')
fig.tight_layout()

tabulate(k_line_pairs, ['Line 1 [Å]', 'Line 2 [Å]', 'Deviation [Å]', 'Radial velocity [km/s]'], tablefmt='html')

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 6))
ca_line_pairs = hd23180.identify_atomic_line(
    CA_I_LINES[0], CA_I_LINES[1],
    axes=axes, draw_expected=True,
    output_difference=True,
    output_radial_velocity=True
)

fig.suptitle(f'Ca I atomic line detection for {hd23180.target}')
fig.tight_layout()

tabulate(ca_line_pairs, ['Line 1 [Å]', 'Line 2 [Å]', 'Deviation [Å]', 'Radial velocity [km/s]'], tablefmt='html')

In [ ]:
def create_line_pair_table_rows(spectrum: Spectrum, line1: AtomicLine, line2: AtomicLine):
    line_pairs = spectrum.identify_atomic_line(line1, line2, output_difference=True, output_radial_velocity=True, tollerance=None)
    extra_info = [spectrum.target]
    return np.column_stack((np.full((line_pairs.shape[0], len(extra_info)), extra_info), line_pairs))

In [ ]:
na_line_pairs_all = np.concatenate([create_line_pair_table_rows(spectrum, NA_I_LINES[0], NA_I_LINES[1]) for spectrum in tqdm(spectra, 'Searching Na I lines')])

headers = ['Target', 'Line 1 [Å]', 'Line 2 [Å]', 'Deviation [Å]', 'Radial velocity [km/s]']
tabulate(na_line_pairs_all, headers, tablefmt='html')

In [ ]:
k_line_pairs_all = np.concatenate([create_line_pair_table_rows(spectrum, K_I_LINES[2], K_I_LINES[3]) for spectrum in tqdm(spectra, 'Searching K I lines')])

headers = ['Target', 'Line 1 [Å]', 'Line 2 [Å]', 'Deviation [Å]', 'Radial velocity [km/s]']
tabulate(k_line_pairs_all, headers, tablefmt='html')

In [ ]:
ca_line_pairs_all = np.concatenate([create_line_pair_table_rows(spectrum, CA_I_LINES[0], CA_I_LINES[1]) for spectrum in tqdm(spectra, 'Searching Ca I lines')])

headers = ['Target', 'Line 1 [Å]', 'Line 2 [Å]', 'Deviation [Å]', 'Radial velocity [km/s]']
tabulate(ca_line_pairs_all, headers, tablefmt='html')

In [ ]:
len(np.unique([spectrum.target for spectrum in spectra]))